In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

In [ ]:
%%writefile intruder_detector.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# intruder Detector script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
mkdir -p $1

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/altera/aocl-pro-rte/aclrte-linux64/
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/computer_vision_sdk/bitstreams/a10_vision_design_bitstreams/5-0_PL1_FP11_ResNet.aocx
fi
    
# Running the intruder detector code
python3 intruder-detector.py -m /opt/intel/computer_vision_sdk/deployment_tools/intel_models/person-vehicle-bike-detection-crossroad-0078/${FP_MODEL}/person-vehicle-bike-detection-crossroad-0078.xml \
                                           -lb resources/labels.txt \
                                           -o $OUTPUT_FILE \
                                           -d $DEVICE \
                                           -l /opt/intel/computer_vision_sdk/deployment_tools/inference_engine/samples/build/intel64/Release/lib/libcpu_extension.so 


In [ ]:
liveQstat()

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te -F "results/core CPU FP32 " -N intrud_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

In [ ]:
print("Submitting a job to an edge compute node with an Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "results/fpga HETERO:FPGA,CPU FP32" -N intrud_fpga
print(job_id_fpga[0])
#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_ncs = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te:intel-ncs -F "results/myriad/ MYRIAD FP16" -N intrud_ncs
print(job_id_ncs[0]) 
#Progress indicators
if job_id_ncs:
    progressIndicator('results/myriad/', 'i_progress_'+job_id_ncs[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub intruder_detector.sh  -l nodes=1:tank-870:e3-1268l-v5 -F "results/xeon/ CPU FP32" -N intrud_xeon 
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "results/gpu/ GPU FP32" -N intrud_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "results/ncs2/ MYRIAD FP16" -N intrud_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_vpu = !qsub intruder_detector.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "results/vpu/ HDDL FP16" -N intrud_vpu
print(job_id_vpu[0]) 
#Progress indicators
if job_id_vpu:
    progressIndicator('results/vpu/', 'i_progress_'+job_id_vpu[0]+'.txt', "Inference", 0, 100)

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub intruder_detector.sh -l nodes=1:up-squared -F "results/up2/ GPU FP32" -N intrud_up2
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('results/up2/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
          ['results/core/video1.mp4','results/core/Statistics.mp4'])

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/gpu/video1.mp4','results/gpu/Statistics.mp4'])

In [ ]:
videoHTML('IEI Tank + IEI Mustang-V100-MX8 (Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))', 
          ['results/vpu/video1.mp4','results/vpu/Statistics.mp4'])

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)', 
          ['results/up2/video1.mp4','results/up2/Statistics.mp4'])

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel NCS2', 
          ['results/ncs2/video1.mp4','results/ncs2/Statistics.mp4'])

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel Movidius NCS', 
          ['results/myriad/video1.mp4','results/myriad/Statistics.mp4'])

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)', 
          ['results/xeon/video1.mp4','results/xeon/Statistics.mp4'])

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)', 
          ['results/fpga/video1.mp4','results/fpga/Statistics.mp4'])